In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("msambare/fer2013")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fer2013


In [2]:
!pip install tensorflow tensorflow-federated tensorflow-privacy

In [3]:
# Install compatible NumPy version
!pip install numpy==1.25.2

import os
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
from glob import glob
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Define constants
IMG_SIZE = 48
BATCH_SIZE = 32
NUM_CLASSES = 7
DATA_DIR = "/kaggle/input/fer2013/train"
EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Load and preprocess images
def load_images_from_folder(folder, label):
    image_paths = glob(os.path.join(folder, '*.jpg'))
    images = []
    labels = []
    for img_path in image_paths:
        img = Image.open(img_path).convert('L')  # Convert to grayscale
        img = img.resize((IMG_SIZE, IMG_SIZE))
        images.append(np.array(img))
        labels.append(label)
    return np.array(images), np.array(labels)

def preprocess_dataset():
    all_images = []
    all_labels = []
    for idx, emotion in enumerate(EMOTIONS):
        images, labels = load_images_from_folder(os.path.join(DATA_DIR, emotion), idx)
        all_images.append(images)
        all_labels.append(labels)
    X = np.concatenate(all_images)
    y = np.concatenate(all_labels)
    X = X.reshape(-1, IMG_SIZE, IMG_SIZE, 1).astype("float32") / 255.0
    y = tf.keras.utils.to_categorical(y, NUM_CLASSES)
    return X, y

# Create federated datasets
def create_federated_data(X, y, num_clients=5):
    client_data = []
    size_per_client = len(X) // num_clients
    for i in range(num_clients):
        start = i * size_per_client
        end = (i + 1) * size_per_client if i < num_clients - 1 else len(X)
        client_dataset = tf.data.Dataset.from_tensor_slices((X[start:end], y[start:end]))
        client_dataset = client_dataset.shuffle(100).batch(BATCH_SIZE)
        client_data.append(client_dataset)
    return client_data

# Define Keras model
def create_keras_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    return model

# TFF model function
def model_fn():
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model(
        keras_model=keras_model,
        input_spec=train_data[0].element_spec,
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[tf.keras.metrics.CategoricalAccuracy()]
    )

# Load and preprocess data
X, y = preprocess_dataset()

# Create federated datasets
train_data = create_federated_data(X, y, num_clients=5)

# Configure Federated Averaging
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=tff.learning.optimizers.build_sgdm(learning_rate=0.02),
    server_optimizer_fn=tff.learning.optimizers.build_sgdm(learning_rate=1.0)
)

# Initialize federated state
state = iterative_process.initialize()

# Training loop
NUM_ROUNDS = 10
for round_num in range(1, NUM_ROUNDS + 1):
    result = iterative_process.next(state, train_data)
    state = result.state
    train_metrics = result.metrics
    print(f'Round {round_num}, Metrics: {train_metrics}')


2025-05-30 06:15:50.826325: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-30 06:15:50.826407: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-30 06:15:50.826475: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Round 1, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.9765927), ('loss', 0.104720525), ('num_examples', 28709), ('num_batches', 900)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 2, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.97923994), ('loss', 0.07450631), ('num_examples', 28709), ('num_batches', 900)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', OrderedDict([('update_non_finite', 0)]))])
Round 3, Metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([('train', OrderedDict([('categorical_accuracy', 0.979449), ('loss', 0.07257603), ('num_examples', 28709), ('num_batches', 900)]))])), ('aggregator', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('finalizer', Ordered